**Convolutional Neural Network Implementation in TensorFlow 2.0**

![Convolutional Neural Network Architecture](https://miro.medium.com/max/2510/1*vkQ0hXDaQv57sALXAJquxA.jpeg)

**About the dataset**



Let us implement a simple convolutional neural network using TensorFlow 2.0. For this, we will make use of the Fashion MNIST dataset by Zalando (MIT License) which contains 70,000 images (in grayscale) in 10 different categories. The images are 28x28 pixels of individual articles of clothing with values ranging from 0 to 255 as shown below:

![Fashion MNIST dataset](https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/doc/img/fashion-mnist-sprite.png)

Out of the total 70,000 images, 60,000 are used for training and remaining 10,000 for testing. The labels are integer arrays ranging from 0 to 9. The class names are not a part of the dataset and hence we need to include the below mapping while training/prediction:

Label	-> Description

0	-> T-shirt/top

1	-> Trouser

2	-> Pullover

3	-> Dress

4	-> Coat

5	-> Sandal

6	-> Shirt

7	-> Sneaker

8	-> Bag

9	-> Ankle boot


In [1]:
# Create class_names list object for mapping labels to names

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [2]:
# Use the below code to make sure that you select TensorFlow 2.0 in Colab
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [3]:
# Install necessary modules

from __future__ import absolute_import, division, print_function, unicode_literals

# Helper libraries
import numpy as np

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras as ks

# Validating the TensorFlow version
print(tf.__version__)

2.2.0


In [4]:
# Load the Fashion MNIST dataset

mnist_fashion = ks.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = mnist_fashion.load_data()

**Data Exploration**

In [5]:
# Shape of Training and Test Set

print('Training Dataset Shape: {}'.format(training_images.shape))
print('No. of Training Dataset Labels: {}'.format(len(training_labels)))
print('Test Dataset Shape: {}'.format(test_images.shape))
print('No. of Test Dataset Labels: {}'.format(len(test_labels)))

Training Dataset Shape: (60000, 28, 28)
No. of Training Dataset Labels: 60000
Test Dataset Shape: (10000, 28, 28)
No. of Test Dataset Labels: 10000


**Data Preprocessing**

As the pixel values range from 0 to 255, we have to scale these values to a range of 0 to 1 before feeding them to the model. We can scale these values (both for training and test datasets) by dividing the values by 255:

In [6]:
training_images = training_images / 255.0

test_images = test_images / 255.0

Reshaping the Training and Test dataset by reshaping the matrices into 28x28x1 array: 

In [7]:
training_images = training_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

In [8]:
# Shape of Training and Test Set after applying reshape

print('Training Dataset Shape: {}'.format(training_images.shape))
print('No. of Training Dataset Labels: {}'.format(len(training_labels)))
print('Test Dataset Shape: {}'.format(test_images.shape))
print('No. of Test Dataset Labels: {}'.format(len(test_labels)))

Training Dataset Shape: (60000, 28, 28, 1)
No. of Training Dataset Labels: 60000
Test Dataset Shape: (10000, 28, 28, 1)
No. of Test Dataset Labels: 10000


**Model Building**

We will be using the keras implementation to build the different layers of a CNN. We will keep it simple by having only 2 layers. 

**First Layer - Convolutional layer with ReLU activation function:** This layer takes the 2D array (28x28 pixels) as input. We will take 50 convolutional kernels (filters) of shape 3x3 pixels, output of whose will be passed to a ReLU activation function before it is passed to the next layer.

In [9]:
cnn_model = ks.models.Sequential()
cnn_model.add(ks.layers.Conv2D(40, (3, 3), activation='relu', input_shape=(28, 28, 1), name='Convolutional_layer'))

**Second Layer - Pooling layer:** This layer takes the 50 26x26 2D arrays as input and transforms them into the same number (50) of arrays with dimensions half of that of the original (i.e. from 26x26 to 13x13 pixels)

In [10]:
cnn_model.add(ks.layers.MaxPooling2D((2, 2), name='Maxpooling_2D'))

**Third Layer - Fully Connected layer:** This layer takes the 50 13x13 2D arrays as input and transforms them into a 1D array of 8450 elements (50x13x13). These 8450 input elements are passed through a fully connected neural network which gives out the probability scores for each of the 10 output labels (at the output layer)

In [11]:
cnn_model.add(ks.layers.Flatten(name='Flatten'))
cnn_model.add(ks.layers.Dense(50, activation='relu', name='Hidden_layer'))
cnn_model.add(ks.layers.Dense(10, activation='softmax', name='Output_layer'))

We can check the details of different layers built in the CNN model by using the summary method as shown below:

In [12]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Convolutional_layer (Conv2D) (None, 26, 26, 40)        400       
_________________________________________________________________
Maxpooling_2D (MaxPooling2D) (None, 13, 13, 40)        0         
_________________________________________________________________
Flatten (Flatten)            (None, 6760)              0         
_________________________________________________________________
Hidden_layer (Dense)         (None, 50)                338050    
_________________________________________________________________
Output_layer (Dense)         (None, 10)                510       
Total params: 338,960
Trainable params: 338,960
Non-trainable params: 0
_________________________________________________________________


Now, we will use an optimization function with the help of compile method. An Adam optimizer with objective function as sparse_categorical_crossentropy which optimzes for the accuracy metric can be built as follows:

In [13]:
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

**Model Training**

In [14]:
cnn_model.fit(training_images, training_labels, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.4356 - accuracy: 0.8454
Epoch 2/10
1875/1875 [==============================] - 53s 28ms/step - loss: 0.3030 - accuracy: 0.8911
Epoch 3/10
1875/1875 [==============================] - 56s 30ms/step - loss: 0.2596 - accuracy: 0.9054
Epoch 4/10
1875/1875 [==============================] - 56s 30ms/step - loss: 0.2322 - accuracy: 0.9152
Epoch 5/10
1875/1875 [==============================] - 57s 31ms/step - loss: 0.2088 - accuracy: 0.9232
Epoch 6/10
1875/1875 [==============================] - 55s 29ms/step - loss: 0.1923 - accuracy: 0.9293
Epoch 7/10
1875/1875 [==============================] - 58s 31ms/step - loss: 0.1742 - accuracy: 0.9352
Epoch 8/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.1598 - accuracy: 0.9412
Epoch 9/10
1875/1875 [==============================] - 57s 30ms/step - loss: 0.1453 - accuracy: 0.9464
Epoch 10/10
1875/1875 [==============================] - 58s 31m

**Model Evaluation**

1. Training Evaluation

In [15]:
training_loss, training_accuracy = cnn_model.evaluate(training_images, training_labels)

print('Training Accuracy {}'.format(round(float(training_accuracy), 2)))

1875/1875 [==============================] - 29s 16ms/step - loss: 0.1167 - accuracy: 0.9576
Training Accuracy 0.96


# RLE: Saving and re-using the model

In [25]:
cnn_model.save('model')

INFO:tensorflow:Assets written to: model/assets


In [27]:
from tensorflow import keras
cnn2_model = keras.models.load_model('model')

2. Test Evaluation

In [28]:
test_loss, test_accuracy = cnn2_model.evaluate(test_images, test_labels)

print('Test Accuracy {}'.format(round(float(test_accuracy), 2)))

313/313 [==============================] - 5s 16ms/step - loss: 0.2729 - accuracy: 0.9075
Test Accuracy 0.91


From the above evaluation, we see that we were able to achieve around 97% accuracy in Training dataset and around 91% accuracy in Test dataset just with a simple CNN architecture. This goes on to prove that CNNs are powerful algorithms for Image recognition.

# THIS DOES NOT WORK, BTW
code for  pickle method
import pickle
## creating/writing binary file 
with open("pickle_model","wb") as f:
      pickle.dump(model,f)
## reading binary file
with open("pickle_mode","rb") as f:
     pm=pickle.load(f)
pm.predict("enter independent variables")


# the code for joblib method..
similarly the code for saving "model" into a file using joblib method is:<br>
from sklearn.externals import joblib
## creating/writing binary file 
joblib.dump(model,"joblib_model")
## reading binary file
jm=joblib.load("joblib_model")<br>
jm.predict(" enter the independent variables")

In [20]:
import pickle
with open("20201108_cnn_mnist_fashion","wb") as f:
    pickle.dump(cnn_model,f)


TypeError: cannot pickle '_thread.RLock' object

In [18]:
import joblib
joblib.dump(cnn_model,"20201108_cnn_mnist_fashion")

TypeError: cannot pickle '_thread.RLock' object